# Scraping data from Retrosheet.org

The goal is to scrape all the batter and pitcher split data from the website Retrosheet.org

In [48]:
from bs4 import BeautifulSoup;
import requests;
import re;

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; \
    Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'};

initials = [ \
    "AA", "AB", "AC", "AD", "AF", "AG", "AH", "AI", "AK", "AL", "AM", "AN", "AO",\
    "AP", "AQ", "AR", "AS", "AT", "AU", "AV", "AX", "AY", "AZ", "BA", "BE", "BI",\
    "BJ", "BL", "BO", "BR", "BU", "BY", "CA", "CE", "CH", "CI", "CL", "CO", "CR",\
    "CU", "CV", "CY", "CZ", "DA", "DE", "DI", "DL", "DO", "DR", "DU", "DW", "DY",\
    "EA", "EB", "EC", "ED", "EE", "EF", "EG", "EH", "EI", "EK", "EL", "EM", "EN",\
    "EO", "EP", "ER", "ES", "ET", "EU", "EV", "EW", "EX", "EY", "EZ", "FA", "FE",\
    "FI", "FL", "FO", "FR", "FU", "FY", "GA", "GE", "GH", "GI", "GL", "GO", "GR",\
    "GS", "GU", "GW", "GY", "HA", "HE", "HI", "HO", "HR", "HU", "HW", "HY", "IA",\
    "IB", "IG", "IL", "IM", "IN", "IO", "IR", "IS", "IV", "IW", "IZ", "JA", "JE",\
    "JI", "JO", "JU", "KA", "KE", "KH", "KI", "KL", "KM", "KN", "KO", "KR", "KU",\
    "KV", "KW", "KY", "LA", "LE", "LI", "LL", "LO", "LU", "LY", "MA", "MC", "ME",\
    "MI", "ML", "MM", "MO", "MR", "MU", "MY", "NA", "NE", "NG", "NI", "NO", "NU",\
    "NY", "OA", "OB", "OC", "OD", "OE", "OF", "OG", "OH", "OJ", "OK", "OL", "OM",\
    "ON", "OQ", "OR", "OS", "OT", "OU", "OV", "OW", "OX", "OY", "OZ", "PA", "PE",\
    "PF", "PH", "PI", "PL", "PO", "PR", "PU", "PY", "QU", "RA", "RE", "RH", "RI",\
    "RL", "RO", "RU", "RY", "RZ", "SA", "SB", "SC", "SE", "SH", "SI", "SK", "SL",\
    "SM", "SN", "SO", "SP", "SQ", "ST", "SU", "SV", "SW", "SY", "SZ", "TA", "TE",\
    "TH", "TI", "TO", "TR", "TS", "TU", "TW", "TY", "UC", "UE", "UG", "UH", "UJ",\
    "UL", "UM", "UN", "UP", "UR", "US", "UT", "VA", "VE", "VI", "VO", "VU", "WA",\
    "WE", "WH", "WI", "WO", "WR", "WU", "WY", "YA", "YD", "YE", "YI", "YN", "YO",\
    "YU", "YV", "ZA", "ZD", "ZE", "ZI", "ZM", "ZO", "ZU", "ZW", "ZY"];
# len = 271
# Might not be a good implementation, since new initials will be added in the future.
# Better to be able to fetch this list automatically.

# Generate a list of pages containing the player pages grouped by initials
player_ini_page = [f"https://www.retrosheet.org/boxesetc/MISC/PLD_{init}.htm" for init in initials];
retro_page_affix = "https://www.retrosheet.org/boxesetc/"

player_urls = [];
# Scrape the urls for each player.
for url in player_ini_page:
    response = requests.get(url, headers=headers);
    text = BeautifulSoup(response.text, 'html.parser');
    
    if response.status_code != 200:
        raise Exception(f"The status code is not 200! It is {response.status_code}.");
    
    player_list = text.find_all("pre")[0].find_all('a')
    player_urlt = [retro_page_affix + tag.get("href")[3:] for tag in player_list]; # Some alignments
    info_list = [tag.getText() for tag in player_list];
    info_list = [[int(info[0:4]), int(info[5:9]), info[10:].split(',')[0]] for info in info_list];

    player_urls.extend(list(map(lambda x, y: x+[y], info_list, player_urlt)));



## Saving the player url data to Pandas dataframe and a .csv file.

In [58]:
import pandas as pd;
import numpy as np;

df_player_urls = pd.DataFrame(np.array(player_urls), columns = ["start_season", "end_season", "Name", "Url"]);
df_player_urls

,start_season,end_season,Name,Url
0,2004,2015,David Aardsma,https://www.retrosheet.org/boxesetc/A/Paardd00...
1,1954,1976,Hank Aaron,https://www.retrosheet.org/boxesetc/A/Paaroh10...
2,1962,1971,Tommie Aaron,https://www.retrosheet.org/boxesetc/A/Paarot10...
3,1977,1990,Don Aase,https://www.retrosheet.org/boxesetc/A/Paased00...
4,2001,2006,Andy Abad,https://www.retrosheet.org/boxesetc/A/Pabada00...
...,...,...,...,...
19914,1957,1961,Frank Zupo,https://www.retrosheet.org/boxesetc/Z/Pzupof10...
19915,1982,1991,Paul Zuvella,https://www.retrosheet.org/boxesetc/Z/Pzuvep00...
19916,1951,1959,George Zuverink,https://www.retrosheet.org/boxesetc/Z/Pzuveg10...
19917,1910,1916,Dutch Zwilling,https://www.retrosheet.org/boxesetc/Z/Pzwild10...


In [59]:
df_player_urls.to_csv("players_url.csv");

In [109]:
df_player_urls["start_season"].apply(int);


## Parsing the url page

In [271]:
df_player_yearly_stats = pd.DataFrame(columns = ["ID", "Name", "Season", "Team", "split_url"]);
df_player_career_stats = pd.DataFrame(columns = ["ID", "Name", "split_url"]);

In [274]:
for ind in df_player_urls.index:
    player_name = df_player_urls.loc[ind, "Name"];
    url = df_player_urls.loc[ind, "Url"];
    
    response = requests.get(url, headers=headers);
    text = BeautifulSoup(response.text, 'html.parser');
    
    if response.status_code != 200:
        raise Exception(f"The status code is not 200! It is {response.status_code}.");
        
    a = text.findAll("pre");
    for ta in a:
        if ta.get_text().find("Batting Record") != -1:
            print("Found batting record for " + player_name + ".");
            break;

    batting_links = ta.findAll("a");

    links_counter = False;
    career_list = [];
    split_link = "";
    for i in range(len(batting_links)):
        if batting_links[i].string == "Splits":
            links_counter = True;
            split_link = retro_page_affix + batting_links[i].get("href")[3:];
        elif batting_links[i].string.isnumeric() and links_counter:
            split_year = int(batting_links[i].string);
            print(split_year);
        elif links_counter:
            links_counter = False;
            career_list.append([ind, player_name, split_year, batting_links[i].string, split_link]);
    
    df_player_yearly_stats = df_player_yearly_stats.append(pd.DataFrame(career_list[0:], columns = df_player_yearly_stats.columns));
    if split_link != "":
        df_player_career_stats.loc[len(df_player_career_stats)] = [ind, player_name, split_link];


Found batting record for David Aardsma.
2006
2008
2015
Found batting record for Hank Aaron.
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
Found batting record for Tommie Aaron.
1962
1963
1965
1968
1969
1970
1971
Found batting record for Don Aase.
1989
Found batting record for Andy Abad.
2001
2003
2006
Found batting record for Fernando Abad.
2010
2012
2016
Found batting record for John Abadie.
Found batting record for Ed Abbaticchio.
1903
1904
1905
1907
1908
1909
1910
1910
Found batting record for Bert Abbey.
Found batting record for Charlie Abbey.
Found batting record for Dan Abbott.
Found batting record for Fred Abbott.
1903
1904
1905
Found batting record for Glenn Abbott.
Found batting record for Jeff Abbott.
1997
1998
1999
2000
2001
Found batting record for Jim Abbott.
1999
Found batting record for Kurt Abbott.
1993
1994
1995
1996
1997
1998
1998
1999
2000
2001
Found batting record for Kyle Abbott.
1992
1995
Found b

Found batting record for Kurt Ainsworth.
2002
2003
Found batting record for Raleigh Aitchison.
1914
1915
Found batting record for George Aiton.
1912
Found batting record for John Ake.
Found batting record for Jack Aker.
1964
1965
1966
1967
1968
1969
1969
1970
1971
1972
1973
1974
1974
Found batting record for Darrel Akerfelds.
1990
1991
Found batting record for Bill Akers.
1929
1930
1931
1932
Found batting record for Jerry Akers.
1912
Found batting record for Keegan Akin.
Found batting record for Shogo Akiyama.
2020
Found batting record for R.J. Alaniz.
2019
Found batting record for Gibson Alba.
Found batting record for Jonathan Albaladejo.
Found batting record for Joe Albanese.
Found batting record for Jose Alberro.
Found batting record for Andrew Albers.
2017
Found batting record for Matt Albers.
2006
2007
2009
2016
Found batting record for Hanser Alberto.
2015
2016
2018
2019
2020
Found batting record for Butch Alberts.
1978
Found batting record for Cy Alberts.
1910
Found batting reco

Found batting record for Matty Alou.
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1972
1973
1973
1974
Found batting record for Moises Alou.
1990
1990
1992
1993
1994
1995
1996
1997
1998
2000
2001
2002
2003
2004
2005
2006
2007
2008
Found batting record for Whitey Alpermann.
1906
1907
1908
1909
Found batting record for Dell Alston.
1977
1978
1978
1979
1980
Found batting record for Garvin Alston.
1996
Found batting record for Tom Alston.
1954
1955
1956
1957
Found batting record for Walter Alston.
1936
Found batting record for Porfi Altamirano.
1982
1983
1984
Found batting record for Dan Altavilla.
Found batting record for Ernie Alten.
1920
Found batting record for Jesse Altenburg.
1916
1917
Found batting record for Aaron Altherr.
2014
2015
2016
2017
2018
2019
2019
2019
Found batting record for Dave Altizer.
1906
1907
1908
1908
1909
1910
1911
Found batting record for George Altman.
1959
1960
1961
1962
1963
1964
1965
1966
1967
Found batting record for Joe Altobelli.
1955


Found batting record for Stan Andrews.
1939
1940
1944
1945
1945
Found batting record for Wally Andrews.
Found batting record for Matt Andriese.
2015
2016
2018
2019
Found batting record for Bill Andrus.
1931
1937
Found batting record for Elvis Andrus.
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Found batting record for Fred Andrus.
Found batting record for Wyman Andrus.
Found batting record for Joaquin Andujar.
1976
1977
1978
1979
1980
1981
1981
1982
1983
1984
1985
1988
Found batting record for Luis Andujar.
Found batting record for Miguel Andujar.
2017
2018
2019
2020
Found batting record for Norm Angelini.
1972
Found batting record for Matt Angle.
2011
Found batting record for Tom Angley.
1929
Found batting record for Pat Ankenman.
1936
1943
1944
Found batting record for Rick Ankiel.
1999
2000
2001
2004
2007
2008
2009
2010
2010
2011
2012
2013
2013
Found batting record for Dean Anna.
2014
2015
Found batting record for Bill Annis.
Found batting record for Cap Anson.
1871


Found batting record for Joe Astroth.
1945
1946
1949
1950
1951
1952
1953
1954
1955
1956
Found batting record for Willians Astudillo.
2018
2019
2020
Found batting record for Carlos Asuaje.
2016
2017
2018
Found batting record for Scott Atchison.
2007
Found batting record for Justin Atchley.
2001
Found batting record for Charlie Atherton.
Found batting record for Keith Atherton.
1983
Found batting record for Luis Atilano.
2010
Found batting record for Garrett Atkins.
2003
2004
2005
2006
2007
2008
2009
2010
Found batting record for James Atkins.
1950
1952
Found batting record for Mitch Atkins.
2010
Found batting record for Tommy Atkins.
1909
1910
Found batting record for Al Atkinson.
Found batting record for Bill Atkinson.
1977
1978
1979
Found batting record for Ed Atkinson.
Found batting record for Harry Atkinson.
Found batting record for Lefty Atkinson.
1927
Found batting record for Dick Attreau.
1926
1927
Found batting record for Toby Atwell.
1952
1953
1953
1954
1955
1956
1956
Found bat

...

Found batting record for Johnny Werts.
1926
1927
1928
1929
Found batting record for Bill Wertz.
Found batting record for Dwight Wertz.
1914
Found batting record for Vic Wertz.
1947
1948
1949
1950
1951
1952
1952
1953
1954
1954
1955
1956
1957
1958
1959
1960
1961
1961
1962
1963
1963
Found batting record for Jim Wessinger.
1979
Found batting record for Barry Wesson.
2002
2003
Found batting record for Billy West.
1874
Found batting record for Buck West.
Found batting record for David West.
1988
1989
1993
1994
1995
1996
Found batting record for Dick West.
1938
1939
1940
1941
1942
1943
Found batting record for Frank West.
Found batting record for Hi West.
1905
1911
Found batting record for Lefty West.
1944
1945
Found batting record for Matt West.
Found batting record for Max West.
1938
1939
1940
1941
1942
1946
1946
1948
Found batting record for Max West.
1928
1929
Found batting record for Sam West.
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1938
1939
1940
1941
1942
Found batt

Found batting record for Art Whitney.
Found batting record for Frank Whitney.
Found batting record for Jim Whitney.
Found batting record for Pinky Whitney.
1928
1929
1930
1931
1932
1933
1933
1934
1935
1936
1936
1937
1938
1939
Found batting record for Bill Whitrock.
Found batting record for Ed Whitson.
1977
1978
1979
1979
1980
1981
1983
1984
1986
1987
1988
1989
1990
1991
Found batting record for Ernie Whitt.
1976
1977
1978
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
Found batting record for Walt Whittaker.
Found batting record for Possum Whitted.
1912
1913
1914
1914
1915
1916
1917
1918
1919
1919
1920
1921
1922
Found batting record for Rowan Wick.
Found batting record for Kevin Wickander.
1993
Found batting record for Bob Wicker.
1901
1902
1903
1903
1904
1905
1906
1906
Found batting record for Floyd Wicker.
1968
1969
1970
1971
1971
Found batting record for Kemp Wicker.
1936
1937
1941
Found batting record for Dave Wickersham.
1960
1961
1962
1963
1964
1965
1966
1967
1968
19

Found batting record for Matt Williams.
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
Found batting record for Matt Williams.
Found batting record for Matt Williams.
2000
Found batting record for Mike Williams.
1992
1993
1994
1995
1996
1998
1999
2000
2002
Found batting record for Mitch Williams.
1989
1990
1991
1992
1993
Found batting record for Mutt Williams.
1913
Found batting record for Nick Williams.
2017
2018
2019
Found batting record for Otto Williams.
1902
1903
1903
1904
1906
Found batting record for Papa Williams.
1945
Found batting record for Pop Williams.
1902
1903
1903
1903
Found batting record for Randy Williams.
2005
Found batting record for Reggie Williams.
1992
1995
1998
1999
Found batting record for Reggie Williams.
1985
1986
1987
1988
Found batting record for Rick Williams.
1978
1979
Found batting record for Rinaldo Williams.
1914
Found batting record for Rip Williams.
1911
1912
1913
1914
1915
1916
1918
Found batting record for Sha

Found batting record for Willie Wilson.
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
Found batting record for Zeke Wilson.
Found batting record for Hal Wiltse.
1926
1927
1928
1928
Found batting record for Hooks Wiltse.
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
Found batting record for Snake Wiltse.
1901
1901
1902
1902
1903
Found batting record for Alex Wimmers.


ConnectionError: HTTPSConnectionPool(host='www.retrosheet.org', port=443): Max retries exceeded with url: /boxesetc/W/Pwince101.htm (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000168DD4BC6A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [275]:

df_player_career_stats

,ID,Name,split_url
0,0,David Aardsma,https://www.retrosheet.org/boxesetc/A/Jaardd00...
1,1,Hank Aaron,https://www.retrosheet.org/boxesetc/A/Jaaroh10...
2,2,Tommie Aaron,https://www.retrosheet.org/boxesetc/A/Jaarot10...
3,3,Don Aase,https://www.retrosheet.org/boxesetc/A/Jaased00...
4,4,Andy Abad,https://www.retrosheet.org/boxesetc/A/Jabada00...
...,...,...,...
15631,19436,Walter Wilson,https://www.retrosheet.org/boxesetc/1945/Jwils...
15632,19437,Willie Wilson,https://www.retrosheet.org/boxesetc/W/Jwilsw00...
15633,19439,Hal Wiltse,https://www.retrosheet.org/boxesetc/W/Jwilth10...
15634,19440,Hooks Wiltse,https://www.retrosheet.org/boxesetc/W/Jwilth10...


In [276]:
df_player_yearly_stats

,ID,Name,Season,Team,split_url
0,0,David Aardsma,2006,CHI N,https://www.retrosheet.org/boxesetc/2006/Jaard...
1,0,David Aardsma,2008,BOS A,https://www.retrosheet.org/boxesetc/2008/Jaard...
2,0,David Aardsma,2015,ATL N,https://www.retrosheet.org/boxesetc/2015/Jaard...
0,1,Hank Aaron,1954,MIL N,https://www.retrosheet.org/boxesetc/1954/Jaaro...
1,1,Hank Aaron,1955,MIL N,https://www.retrosheet.org/boxesetc/1955/Jaaro...
...,...,...,...,...,...
0,19441,Snake Wiltse,1901,PIT N,https://www.retrosheet.org/boxesetc/1901/Jwilt...
1,19441,Snake Wiltse,1901,PHI A,https://www.retrosheet.org/boxesetc/1901/Jwilt...
2,19441,Snake Wiltse,1902,PHI A,https://www.retrosheet.org/boxesetc/1902/Jwilt...
3,19441,Snake Wiltse,1902,BAL A,https://www.retrosheet.org/boxesetc/1902/Jwilt...


In [277]:
url = df_player_urls.loc[15273, "Url"];
    
response = requests.get(url, headers=headers);
text = BeautifulSoup(response.text, 'html.parser');

if response.status_code != 200:
    raise Exception(f"The status code is not 200! It is {response.status_code}.");

a = text.findAll("pre");
for ta in a:
    if ta.get_text().find("Batting Record") != -1:
        print("Found batting record for " + player_name + ".");
        break;

batting_links = ta.findAll("a");

Found batting record for Ed Winceniak.


In [270]:
df_player_urls

,start_season,end_season,Name,Url
0,2004,2015,David Aardsma,https://www.retrosheet.org/boxesetc/A/Paardd00...
1,1954,1976,Hank Aaron,https://www.retrosheet.org/boxesetc/A/Paaroh10...
2,1962,1971,Tommie Aaron,https://www.retrosheet.org/boxesetc/A/Paarot10...
3,1977,1990,Don Aase,https://www.retrosheet.org/boxesetc/A/Paased00...
4,2001,2006,Andy Abad,https://www.retrosheet.org/boxesetc/A/Pabada00...
...,...,...,...,...
19914,1957,1961,Frank Zupo,https://www.retrosheet.org/boxesetc/Z/Pzupof10...
19915,1982,1991,Paul Zuvella,https://www.retrosheet.org/boxesetc/Z/Pzuvep00...
19916,1951,1959,George Zuverink,https://www.retrosheet.org/boxesetc/Z/Pzuveg10...
19917,1910,1916,Dutch Zwilling,https://www.retrosheet.org/boxesetc/Z/Pzwild10...
